# 02 - Cloud Build

## Setup

If you haven't already, install the toolkit and dependencies using the [Setup](./00-Setup.ipynb) notebook.

## Build

In [1]:
%reload_ext dotenv
%dotenv

import os

for key, value in os.environ.items():
    print(f"{key}={value}")

GSM_SKIP_SSH_AGENT_WORKAROUND=true
GJS_DEBUG_TOPICS=JS ERROR;JS LOG
LC_ALL=en_US.UTF-8
SNAP_LIBRARY_PATH=/var/lib/snapd/lib/gl:/var/lib/snapd/lib/gl32:/var/lib/snapd/void
WINDOWPATH=2
GJS_DEBUG_OUTPUT=stderr
GNOME_SHELL_SESSION_MODE=ubuntu
SNAP_NAME=pycharm-professional
PATH=/home/evanerwee/anaconda3/envs/graphrag-toolkit6/bin:/home/evanerwee/anaconda3/condabin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin:/snap/bin
CONDA_PREFIX=/home/evanerwee/anaconda3/envs/graphrag-toolkit6
CLUTTER_DISABLE_MIPMAPPED_TEXT=1
LOGNAME=evanerwee
XDG_MENU_PREFIX=gnome-
XDG_CONFIG_DIRS=/etc/xdg/xdg-ubuntu:/etc/xdg
XDG_ACTIVATION_TOKEN=gnome-shell/PyCharm/112630-1-RWLNX01_TIME130085290
BAMF_DESKTOP_FILE_HINT=/var/lib/snapd/desktop/applications/pycharm-professional_pycharm-professional.desktop
XAUTHORITY=/run/user/1000/gdm/Xauthority
CONDA_SHLVL=1
SNAP_ARCH=amd64
XMODIFIERS=@im=ibus
GIO_LAUNCHED_DESKTOP_FILE_PID=115692
SNAP_REVISION=475
GIO_LAUNCHED_DESKTO

In [ ]:
%reload_ext dotenv
%dotenv

import os

from graphrag_toolkit.lexical_graph import LexicalGraphIndex, set_logging_config
from graphrag_toolkit.lexical_graph.storage import GraphStoreFactory
from graphrag_toolkit.lexical_graph.storage import VectorStoreFactory
from graphrag_toolkit.lexical_graph.indexing.load import S3BasedDocs
from graphrag_toolkit.lexical_graph.indexing.build import Checkpoint

set_logging_config('DEBUG')

docs = S3BasedDocs(
    region='us-east-1',
    bucket_name=os.environ['S3_BUCKET_EXTRACK_BUILD_BATCH_NAME'],
    key_prefix='extract-build',
    collection_id='best-practices'
)
checkpoint = Checkpoint('s3-extraction-checkpoint')

graph_store = GraphStoreFactory.for_graph_store(os.environ['GRAPH_STORE'])
vector_store = VectorStoreFactory.for_vector_store(os.environ['VECTOR_STORE'])

graph_index = LexicalGraphIndex(
    graph_store,
    vector_store
)

graph_index.build(docs, checkpoint=checkpoint, show_progress=True)

print('Build complete')